# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [8]:
# imports
import panel as pn
from panel.interact import interact
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv


In [9]:
# Set up Panel Plotly extension
pn.extension('plotly')

In [10]:
# Import hvplot.pandas after pn.extension
# This avoids plotly initialization failure
import hvplot.pandas

In [11]:
# Set up API credentials
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("GGC_MAXPRO")

# Set token using Plotly Express set function
px.set_mapbox_access_token(map_box_api)


# Import Data

In [12]:
# Import the CSVs to Pandas DataFrames
# Read the census data into a Pandas DataFrame
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


In [13]:
# Load neighborhoods coordinates data
file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)
df_neighborhood_locations.head()

,Neighborhood,Lat,Lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [14]:
# Define Panel Visualization Functions

"""Housing Units Per Year."""
def housing_units_per_year():
    
    housing_units = sfo_data['housing_units'].groupby('year').mean()
    df_housing_units = housing_units.to_frame().reset_index()

    return px.bar(
        df_housing_units, 
        x=df_housing_units['year'],
        y=df_housing_units['housing_units'],
        title=(f'Housing Units in SFO from {housing_units.index[0]} to {housing_units.index[-1]}')
    ).update_yaxes(range=[(housing_units.min()-housing_units.std()), (housing_units.max()+housing_units.std())])


""" Master Data for average_gross_rent & average_sales_price """
def data_avg_rent_price():
    
    saleprice_grossrent = sfo_data['sale_price_sqr_foot'].groupby('year').mean(), sfo_data['gross_rent'].groupby('year').mean()
    col_names = ["sale_price_sqr_foot", "gross_rent"]
    saleprice_grossrent = pd.concat(saleprice_grossrent, axis=1, keys=col_names).reset_index()
    
    return saleprice_grossrent


"""Average Gross Rent in San Francisco Per Year."""
def average_gross_rent():  
    saleprice_grossrent = data_avg_rent_price()

    return px.line(
        saleprice_grossrent, 
        x=saleprice_grossrent['year'],
        y=saleprice_grossrent['gross_rent'],
        title=(f'Average Gross Rent in SFO')
    )


"""Average Sales Price Per Year."""
def average_sales_price():
    saleprice_grossrent = data_avg_rent_price()


    return px.line(
        saleprice_grossrent, 
        x=saleprice_grossrent['year'],
        y=saleprice_grossrent['sale_price_sqr_foot'],
        title=(f'Average Sales Price per Square Foot in SFO')
    )


"""Average Prices by Neighborhood."""
def average_price_by_neighborhood():

    avgprice_neighborhood = sfo_data[['neighborhood','sale_price_sqr_foot']].groupby([sfo_data.index, 'neighborhood']).mean().reset_index()
    avgprice_neighborhood.head()

    return avgprice_neighborhood.hvplot(kind='line', x='year', y='sale_price_sqr_foot', groupby='neighborhood', dynspread=True, datashade=False).opts(framewise=True)



""" Master Data for top_most_expensive_neighborhoods(), parallel_coordinates(), parallel_categories() """
def data_top10_neighborhoods():
    
    top10 = sfo_data.groupby('neighborhood').mean().sort_values(by=['sale_price_sqr_foot'],axis=0,ascending=False).reset_index()
    top10 = top10[0:10]
    
    return top10


"""Top 10 Most Expensive Neighborhoods."""
def top_most_expensive_neighborhoods():
    top10 = data_top10_neighborhoods()

    return px.bar(
        top10, 
        x=top10['neighborhood'],
        y=top10['sale_price_sqr_foot'],
        title=(f'top 10 neighborhoods in SFO from "{top10.iloc[0]["neighborhood"]}" to "{top10.iloc[9]["neighborhood"]}"')
    )



"""Parallel Coordinates Plot."""
def parallel_coordinates():
    top10 = data_top10_neighborhoods()

    return px.parallel_coordinates(top10, color='sale_price_sqr_foot')



"""Parallel Categories Plot."""
def parallel_categories():
    top10 = data_top10_neighborhoods()

    return px.parallel_categories(
        top10,
        dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={
         "neighborhood": "Neighborhood",
            "sale_price_sqr_foot": "Avg.SalePrice SqrFoot",
            "housing_units": "Housing Units",
            "gross_rent": "Gross Rent",
            }
        )



"""Neighborhood Map"""
def neighborhood_map():
    
    neighborhood_mean = sfo_data.groupby('neighborhood').mean().reset_index()
    df_mean_loc = pd.concat([df_neighborhood_locations, neighborhood_mean], axis = 1).drop(['neighborhood'], axis=1)

    return px.scatter_mapbox(
        df_mean_loc,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="gross_rent",
        title="Average Sale Price per Square Foot and Gross Rent in San Francisco",
        zoom=12
    )



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [15]:
row_mkt_evol_fco = pn.Row("## San Francisco Market Evolution since 2010", housing_units_per_year())
row_mkt_evol_fco.append(average_gross_rent())
row_mkt_evol_fco.append(average_sales_price())


In [16]:
col_neighborhoods_fco = pn.Column("## Neighborhoods Analysis", average_price_by_neighborhood())
col_neighborhoods_fco.append(top_most_expensive_neighborhoods())


In [17]:
row_multi_dim = pn.Row("## Multi-Dimension Analysis (Sales-Price, Housing Units, Gross Rent", parallel_coordinates())
row_multi_dim.append(parallel_categories())


In [18]:
col_neighborhoods_maps= pn.Column("## Interactive Maps", neighborhood_map())


In [19]:
# Create tabs
sfo_dashboard = pn.Tabs(
    ("SFO Market Evolution", row_mkt_evol_fco),
    ("Neighborhoods", col_neighborhoods_fco),
    ("Multi-Dimension View", row_multi_dim),
    ("Interactive Map", col_neighborhoods_maps),
)

## Serve the Panel Dashboard

In [20]:
sfo_dashboard.servable()


Tabs
    [0] Row
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
        [3] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] Plotly(Figure)
    [2] Row
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)